In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import requests
from torch.utils.data import Dataset, DataLoader
import zipfile
import pandas as pd
import os
import torch
import io
import numpy as np
from torchvision.io import read_image
from torchvision import transforms
import matplotlib.pyplot as plt
from torchvision.models import resnet50, ResNet50_Weights
import torch.nn as nn

In [ ]:
class MelanomaDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.landmark_frame = pd.read_csv(csv_file,names=['img_name', 'class'])
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.landmark_frame)

        # Normalization of images as described in the paper
    def normalize(self,image):
        image[0,:,:] = (image[0,:,:]-torch.mean(image[0,:,:].float()))/torch.std(image[0,:,:].float())
        image[1,:,:] = (image[1,:,:]-torch.mean(image[1,:,:].float()))/torch.std(image[1,:,:].float())
        image[2,:,:] = (image[2,:,:]-torch.mean(image[2,:,:].float()))/torch.std(image[2,:,:].float())
        return image
        
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        classes = {'malignant':1.0, 'benign':0.0}
        img_name = os.path.join(self.root_dir, self.landmark_frame.iloc[idx, 0]+'.jpg').replace('\\','/')
        label = self.landmark_frame.iloc[idx, 1]
        image = read_image(img_name)
        
        if self.transform:
            sample = [image, classes[label]]
            sample[0] = self.transform(sample[0])
        else:
            sample = [image,label]
            transform2 = transforms.Compose([
                transforms.ToPILImage(),
                transforms.Resize((224,224)),
                transforms.ToTensor()
                ])
            sample[0] = transform2(sample[0])

        sample[0] = self.normalize(sample[0])
        return sample


In [30]:
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224,224)),
    transforms.RandomRotation(degrees=(-180, 180)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.ToTensor()
])

In [31]:
melanoma_dataset = MelanomaDataset('/kaggle/input/isbi2016-isic-part3/ISBI2016_ISIC_Part3_Training_GroundTruth.csv', '/kaggle/input/isbi2016-isic-part3/ISBI2016_ISIC_Part3_Training_Data/ISBI2016_ISIC_Part3_Training_Data',transform=transform)

In [32]:
batch_size = 64
dataloader = DataLoader(melanoma_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# model = resnet50(weights = ResNet50_Weights.IMAGENET1K_V2)
# model.fc = nn.Linear(2048,1, bias=False)
# model = model.to(device)

In [ ]:
PATH = '/kaggle/input/melanoma_resnet50/pytorch/default/1/MelanomaResNet50'
model = resnet50()
model.fc = nn.Linear(2048, 1, bias=False)
model = model.to(device)
model.load_state_dict(torch.load(PATH, map_location=device))

In [ ]:
# Training the whole model on our dataset
epochs = 100
learning_rate = 1e-5

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

for epoch in range(epochs):
    for idx, (img, label) in enumerate(dataloader):
        img = img.to(device)
        label = torch.Tensor(label)
        label = label.unsqueeze(1).to(device)

        # forward pass
        y_pred = model(img)
        loss = criterion(y_pred, label)

        # backward pass
        loss.backward()

        # weight update
        optimizer.step()

        # making gradint zero
        optimizer.zero_grad()
    print(epoch)

In [35]:
PATH = '/kaggle/working/MelanomaResNet50_v2'
torch.save(model.state_dict(),PATH)

In [36]:
model = resnet50()
model.fc = nn.Linear(2048, 1, bias=False)
model = model.to(device)
model.load_state_dict(torch.load(PATH, map_location=device))

<All keys matched successfully>

In [37]:
melanoma_dataset_test = MelanomaDataset('/kaggle/input/isbi2016-isic-part3/ISBI2016_ISIC_Part3_Test_GroundTruth.csv', '/kaggle/input/isbi2016-isic-part3/ISBI2016_ISIC_Part3_Test_Data/ISBI2016_ISIC_Part3_Test_Data')
# batch_size = 8
dataloader_test = DataLoader(melanoma_dataset_test, batch_size=batch_size, shuffle=False)

In [40]:
# Accuracy 
correct = 0
total = 0

with torch.no_grad():
    for (image, label) in dataloader_test:
        image = image.to(device)
        label = torch.Tensor(label).unsqueeze(1).to(device)
        
        output = model(image)
        prediction = []
        for i in output:
            if i>0:
                prediction.append(1)
            else:
                prediction.append(0)
        
#         prediction = torch.max(output.data,1)
        prediction = torch.Tensor(prediction).unsqueeze(1).to(device)
        total += label.size(0)
        correct +=  (prediction==label).sum().item()

print(f'The Accuracy of the model is {100*correct/total}')

The Accuracy of the model is 92.0
